In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-opt")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-187199
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-187199


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cluster_name = "criscluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil
from scipy.stats import loguniform
from azureml.train.hyperdrive import BanditPolicy

# Parameter sampler suitable for logistic regression
ps = RandomParameterSampling(
    {
#        '--C' : loguniform(1e-5, 100),
        '--C': choice(0.1,1.0,10),
        '--max_iter': choice(100,200,300)
    }
)


# Specify a Policys
policy = BanditPolicy(slack_factor = 0.1,
            evaluation_interval = 1,
            delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

script_folder = "./training"    
    
# Copy the train.py into the script folder
shutil.copy('train.py', script_folder)

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
        script = 'train.py',
        compute_target=cluster_name,
        environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=6,
                                    max_concurrent_runs=4)


In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3bd2043e-edf3-4d10-96fa-ab098d563e28
Web View: https://ml.azure.com/runs/HD_3bd2043e-edf3-4d10-96fa-ab098d563e28?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-187199/workspaces/quick-starts-ws-187199&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-21T09:06:15.101420][API][INFO]Experiment created<END>\n""<START>[2022-02-21T09:06:15.972163][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-21T09:06:16.534269][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_3bd2043e-edf3-4d10-96fa-ab098d563e28
Web View: https://ml.azure.com/runs/HD_3bd2043e-edf3-4d10-96fa-ab098d563e28?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-187199/workspaces/quick-starts-ws-187199&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_3bd2043e-edf3-4d10-96fa-ab098d563e28',
 'target': 'criscluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-21T09:06:14.865861Z',
 'endTimeUtc': '2022-02-21T09:10:51.708557Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9942fdc0-78c1-4a37-9497-c2d75f2bf95a',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '9',
  'score': '0.9136570561456753',
  'best_child_run_id': 'HD_3bd2043e-edf3-4d10-96fa-ab098d563e28_5',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187199.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3bd2043e-edf3-4d10-96fa-ab098d563e28/azureml-logs/hyper

In [22]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())
print('File name: ', best_run.get_file_names())

ID :  HD_3bd2043e-edf3-4d10-96fa-ab098d563e28_5
Metrics :  {'Regularization Strength:': 10.0, 'Max iterations:': 300, 'Accuracy': 0.9136570561456753}
File name:  ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [23]:
best_model = best_run.register_model(model_name = 'model',model_path = './')

In [26]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dataset_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path=dataset_url)

In [27]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='calssification',
    primary_metric='accuracy',
    training_data=train,
    label_column_name=test,
    n_cross_validations=c)

In [2]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

automl_run,fitted_automl_model = automl_run.get_output()
print(fitted_automl_model)

joblib.dump(fitted_automl_model, "automl_model.joblib")